# Transformers

## Reference 
- https://huggingface.co/docs/transformers/v4.18.0/en/pipeline_tutorial
- https://transformers.run/intro/2021-12-08-transformers-note-1/
- https://github.com/huggingface/transformers/issues/26959
- https://github.com/huggingface/transformers/issues/26959

## A. KERNEL

In [ ]:
# kernel Image_pytorch_2.1.0-cuda11.8-cudnn8-devel
# ipykernel 3.10
#!/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity exec --nv -B /work /work/u00cjz00/nvidia/pytorch_2.0.1-cuda11.7-cudnn8-runtime.sif pip install -q ipykernel

## B. SET PATH

In [1]:
# 新增可讀取套件安裝位置
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=Add_Binarry_Path+':'+os.environ['PATH']

In [4]:
# 安裝套件 GPTQ
!pip install xformers optimum bitsandbytes sentence_transformers einops arxiv openai chromadb tiktoken pymupdf langchain chainlit transformers IProgress ipywidgets accelerate -q
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  -q # Use cu117 if on CUDA 11.7
#!pip3 install transformers>=4.32.0 optimum>=1.12.0 -q
#!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ -q # Use cu117 if on CUDA 11.7

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from auto_gptq import AutoGPTQForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationBufferMemory
import torch
import warnings
warnings.filterwarnings('ignore')

## 推理管道 Pipelines for inference
pipeline ()可以使用模型中心的任何模型來推理各種任務，例如文字生成、圖像分割和音訊分類。

### 1. 情緒分析
pipeline 模型會自動完成以下三個步驟：
- 將文字預處理為模型可以理解的格式；
- 將預處理好的文字送入模型；
- 對模型的預測值進行後處理，輸出人類可以理解的格式。

In [6]:
###  model https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english
from transformers import pipeline
generator = pipeline("sentiment-analysis")
generator("I've been waiting for a HuggingFace course my whole life.")
generator(["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9598046541213989},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

### 2. 零訓練樣本分類
零訓練樣本分類pipeline 允許我們在不提供任何標註資料的情況下自訂分類標籤。

In [7]:
# model https://huggingface.co/facebook/bart-large-mnli
from transformers import pipeline
generator = pipeline("zero-shot-classification")
generator("This is a course about the Transformers library",candidate_labels=["education", "politics", "business"])

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445951342582703, 0.1119769886136055, 0.04342784732580185]}

### 3. 文字生成 text-generation

In [8]:
# model https://huggingface.co/gpt2
from transformers import pipeline
generator = pipeline(task="text-generation")
generator("In this course, we will teach you how to")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to connect digital and physical objects together to make your work life more fluid and memorable. A simple task set up using the touch of a button, this course will quickly demonstrate how to get started using touch to'}]

## 這些pipeline 背後做了什麼？
- 預處理(preprocessing)，將原始文字轉換為模型可以接受的輸入格式；
- 將處理好的輸入送入模型；
- 對模型的輸出進行後處理(postprocessing)，將其轉換為人類方便閱讀的格式。

![NN] <img src="https://transformers.run/assets/img/transformers-note-1/full_nlp_pipeline.png" width="1000">

In [ ]:
# model Llama-2-7b-chat-hf
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
MODEL_ID = "/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
generator("In this course, we will teach you how to")

In [5]:
# model Llama-2-7b-chat-hf load_in_4bit
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
MODEL_ID = "/work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
# load model in 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, quantization_config=quantization_config, device_map="auto")
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer,max_length=200)
generator("In this course, we will teach you how to")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[{'generated_text': 'In this course, we will teach you how to create a successful YouTube channel, from finding your niche and creating content to building a community and monetizing your channel.ἱ\n\n\n\n'}]

In [14]:
# model Llama-2-7b-chat-hf GPTQ
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from auto_gptq import AutoGPTQForCausalLM
MODEL_ID = "/work/u00cjz00/slurm_jobs/github/models/Llama-2-7B-Chat-GPTQ"
model = AutoGPTQForCausalLM.from_quantized(MODEL_ID,  device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True, use_safetensors=True, use_triton=False)
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
generator("In this course, we will teach you how to",max_length=200)

Skipping module injection for FusedLlamaMLPForQuantizedModel as currently not supported with use_triton=False.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlama

[{'generated_text': 'In this course, we will teach you how to create a successful and sustainable business by leveraging the power of social media marketing.\nIn this course, you will learn how to:\n1. Identify your target audience and create buyer personas\n2. Develop a social media marketing strategy that aligns with your business goals\n3. Create engaging and shareable content that drives traffic and conversions\n4. Use paid social media advertising to reach a wider audience\n5. Measure and track your social media performance using analytics tools\n6. Optimize your social media campaigns for maximum ROI\n7. Utilize influencer marketing to reach new audiences\n8. Leverage user-generated content to build brand loyalty\n9. Use social media to improve customer service and build brand reputation\n10. Stay up-to-date with the latest social media trends and best practices\nBy the end of this'}]

In [22]:
# model Llama-2-7b-chat-hf GPTQ + configure +  prompt
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from auto_gptq import AutoGPTQForCausalLM
MODEL_ID = "/work/u00cjz00/slurm_jobs/github/models/Llama-2-7B-Chat-GPTQ"
model = AutoGPTQForCausalLM.from_quantized(MODEL_ID,  device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

# Inference can also be done using transformers' pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

prompt = "Tell me about AI"
prompt_template=f'''[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt}[/INST]

'''
print(generator(prompt_template)[0]['generated_text'])

Skipping module injection for FusedLlamaMLPForQuantizedModel as currently not supported with use_triton=False.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlama

[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Tell me about AI[/INST]

Hello! I'm here to assist you with any questions or topics you have in mind. AI (Artificial Intelligence) is a fascinating field that has been rapidly advancing in recent years. AI refers to the ability of machines to perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception.
There are several types of AI, including:
1. Narrow or weak AI: This type of AI is designed to perfo

In [28]:
# model Llama-2-7b-chat-hf GPTQ + configure +  prompt
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from auto_gptq import AutoGPTQForCausalLM
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate
MODEL_ID = "/work/u00cjz00/slurm_jobs/github/models/Llama-2-7B-Chat-GPTQ"
model = AutoGPTQForCausalLM.from_quantized(MODEL_ID,  device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

# Inference can also be done using transformers' pipeline
#generator = pipeline("text-generation",model=model,tokenizer=tokenizer,
#    max_new_tokens=512,
#    do_sample=True,
#    top_p=0.95,
#    top_k=40,
#    repetition_penalty=1.1
#)

generator=pipeline("text-generation", model=model, tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )


llm=HuggingFacePipeline(pipeline=generator, model_kwargs={'temperature':0.7})
llm("In this course, we will teach you how to")

Skipping module injection for FusedLlamaMLPForQuantizedModel as currently not supported with use_triton=False.
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlama

" create a successful online business and make money online through various legitimate means, including:\n\n1. Affiliate marketing\n2. Online tutoring and coaching\n3. Selling digital products and courses\n4. Creating and selling an online service-based business\n5. Building a blog or website and monetizing it through advertising, sponsorships, and affiliate marketing\nWe will cover everything you need to know to get started, from finding your niche and building a website, to marketing and promoting your business and reaching your target audience. By the end of this course, you will have a solid understanding of how to build a successful online business and make money online, and you will be well on your way to achieving your financial goals.\nSo, if you're ready to take your business to the next level and start making money online, enroll in this course today and let's get started!"

In [29]:
# We define our prompt template in the following way:
prompt_template = """<s>[INST] <<SYS>>
{{ You are a helpful AI Assistant}}<<SYS>>
###

Previous Conversation:
'''
{history}
'''

{{{input}}}[/INST]

"""
prompt = PromptTemplate(template=prompt_template, input_variables=['input', 'history'])

In [27]:
# 為了防止上下文緩衝區變得非常大，我們可以使用ConversationBufferWindowMemory. 只需修改定義鏈的程式碼部分即可。然後，再次運行鏈條！
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferWindowMemory(k=5)
llm_chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [30]:
llm_chain.run("Tell me about elephant")

'Of course! Elephants are fascinating creatures that are known for their intelligence, social behavior, and unique characteristics. Here are some interesting facts about elephants:\n\n1. Largest Living Thing: Elephants are the largest living things on Earth, with African elephants weighing up to 6 tons (12,000 pounds) and Asian elephants weighing up to 5 tons (10,000 pounds).\n2. Intelligent Creatures: Elephants are highly intelligent animals that have been observed using tools, solving problems, and even displaying empathy and self-awareness.\n3. Social Behavior: Elephants live in complex societies with strict hierarchies and social bonds. They communicate with each other through a variety of sounds, including trumpets, rumbles, and squeaks.\n4. Long Lifespan: Elephants can live for up to 70 years in the wild and over 80 years in captivity, making them one of the longest-living land animals.\n5. Unique Trunk: Elephants have a highly specialized trunk that is used for breathing, drinki

In [31]:
llm_chain.run("How weight is it?")

"As a helpful AI assistant, I'm happy to provide information on various topics. However, I must inform you that the previous conversation you provided contains 8 points of interesting facts about elephants, and none of them relate to the weight of elephants. Therefore, I cannot provide an answer to your question regarding how much an elephant weighs.\nElephants are indeed massive creatures, with African elephants typically weighing between 5,000 to 14,000 pounds (2,268 to 6,350 kilograms), while Asian elephants typically weigh between 3,000 to 6,000 pounds (1,361 to 2,722 kilograms). If you have any further questions or would like to know more about elephants, please feel free to ask!"

In [32]:
llm_chain.run(" Where can I see it?")

"As a helpful AI assistant, I'm glad you're interested in learning more about elephants! There are several places where you can observe these incredible creatures in person or virtually. Here are some options:\n1. National Zoos and Wildlife Sanctuaries: Many national zoos and wildlife sanctuaries around the world have elephants in their care. These institutions often offer educational programs, exhibits, and conservation initiatives dedicated to protecting elephants and their habitats. Some notable examples include the San Diego Zoo, the Bronx Zoo, and the Elephant Nature Park in Thailand.\n2. Wildlife Conservation Organizations: Several organizations are dedicated to protecting and conserving elephant populations in the wild. For example, the World Wildlife Fund (WWF) and the International Union for Conservation of Nature (IUCN) work to safeguard habitats, monitor elephant populations, and combat poaching. You can visit their websites to learn more about their efforts and how you can 

In [33]:
memory.load_memory_variables({})

{'history': "Human: Tell me about elephant\nAI: Of course! Elephants are fascinating creatures that are known for their intelligence, social behavior, and unique characteristics. Here are some interesting facts about elephants:\n\n1. Largest Living Thing: Elephants are the largest living things on Earth, with African elephants weighing up to 6 tons (12,000 pounds) and Asian elephants weighing up to 5 tons (10,000 pounds).\n2. Intelligent Creatures: Elephants are highly intelligent animals that have been observed using tools, solving problems, and even displaying empathy and self-awareness.\n3. Social Behavior: Elephants live in complex societies with strict hierarchies and social bonds. They communicate with each other through a variety of sounds, including trumpets, rumbles, and squeaks.\n4. Long Lifespan: Elephants can live for up to 70 years in the wild and over 80 years in captivity, making them one of the longest-living land animals.\n5. Unique Trunk: Elephants have a highly speci

### 4. 自動問答
自動問答pipeline 可以根據給定的上下文回答問題，例如：

In [16]:
# model https://huggingface.co/distilbert-base-cased-distilled-squad)
from transformers import pipeline
generator = pipeline("question-answering")
generator(question="Where am I form?",context="My name is Sylvain and I in Taiwan")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.9241172075271606, 'start': 28, 'end': 34, 'answer': 'Taiwan'}

### 5. 自動摘要
自動摘要pipeline 旨在將長文本壓縮成短文本，並且還要盡可能保留原文的主要訊息，例如：

In [17]:
# model https://huggingface.co/sshleifer/distilbart-cnn-12-6
from transformers import pipeline
generator = pipeline("summarization")
generator(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
    """
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil, electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India, as well as other industrial countries in Europe and Asia, continue to encourage and advance engineering .'}]

## 這些pipeline 背後做了什麼？
- 預處理(preprocessing)，將原始文字轉換為模型可以接受的輸入格式；
- 將處理好的輸入送入模型；
- 對模型的輸出進行後處理(postprocessing)，將其轉換為人類方便閱讀的格式。

![alt text](https://transformers.run/assets/img/transformers-note-1/full_nlp_pipeline.png)

### 6. vision_classifier

In [18]:
# model google/vit-base-patch16-224
from transformers import pipeline
vision_classifier = pipeline(task="image-classification")
vision_classifier(
    images="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
)

No model was supplied, defaulted to google/vit-base-patch16-224 and revision 5dca96d (https://huggingface.co/google/vit-base-patch16-224).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'score': 0.4403040111064911, 'label': 'lynx, catamount'},
 {'score': 0.03433402255177498,
  'label': 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor'},
 {'score': 0.03214792534708977,
  'label': 'snow leopard, ounce, Panthera uncia'},
 {'score': 0.023539021611213684, 'label': 'Egyptian cat'},
 {'score': 0.02303403615951538, 'label': 'tiger cat'}]

### 7. audio_classifier

In [ ]:
from transformers import pipeline
audio_classifier = pipeline(
    task="audio-classification", model="ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition"
)
audio_classifier("jfk_moon_speech.wav")